In [1]:
from pathlib import Path
import pyrealsense2 as rs
import numpy as np
from sapien import Pose
from transforms3d.euler import euler2quat
from real_robot.agents import XArm7
from real_robot.sensors.camera import CameraConfig, Camera
from real_robot.utils.visualization import Visualizer
from real_robot.utils.multiprocessing import ctx, SharedObject, start_and_wait_for_process

# device = get_connected_rs_devices('146322076186')
# device = get_connected_rs_devices('146322072630')

agent_proc = ctx.Process(
    target=XArm7, name="XArm7_state_stream",
    args=("192.168.1.229", "pd_ee_delta_pos", "cartesian_online"),
    kwargs=dict(
        with_hand_camera=True,
        run_as_process=True,
    )
)
start_and_wait_for_process(agent_proc, timeout=30)
so_agent_start = SharedObject("start_xarm7_real")
so_agent_start.assign(True)

pose_tcp_cam = Pose(p=[0, 0, 0.177]).inv() * Pose(
    p=[-0.06042734, 0.0175, 0.02915237],
    q=euler2quat(np.pi, -np.pi/2-np.pi/12, np.pi)
) * Pose(p=[0, 0.015, 0])  # camera_color_frame
camera = Camera(
    CameraConfig(
        "hand_camera", None, pose_tcp_cam, config={
            "Color": (848, 480, 60),
            "Depth": (848, 480, 60),
            "Infrared 1": (848, 480, 60),
            "Infrared 2": (848, 480, 60),
        }, preset="Default", parent_pose_so_name="xarm7_real_tcp_pose",
        depth_option_kwargs={rs.option.laser_power: 360}
    ),
)

visualizer = Visualizer(run_as_process=True, stream_camera=True, stream_robot=True)
camera

SDK_VERSION: 1.13.8+patch20231009
SDK_VERSION: 1.13.8+patch20231009
ROBOT_IP: 192.168.1.229, VERSION: v1.12.10, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1302,v1.12.10, TYPE1300: [1, 1]
change protocol identifier to 3


[2023-10-20 17:46:12,359] [XArm7] [xarm.py:509] [INFO] Running <XArm7: ip=192.168.1.229, control_mode=pd_ee_delta_pos, motion_mode=cartesian_online, with_hand_camera=True> as a separate process
[2023-10-20 17:46:12,584] [realsense.py] [realsense.py:52] [INFO] Found Intel RealSense D435 (S/N: 146322076186 FW: 5.15.1 on USB 3.2)
[2023-10-20 17:46:12,585] [realsense.py] [realsense.py:57] [INFO] Found 1 devices
[2023-10-20 17:46:12,931] [RSDevice] [realsense.py:316] [INFO] Loaded "Default" preset for <RSDevice: hand_camera (Intel RealSense D435, S/N: 146322076186)>
[2023-10-20 17:46:12,932] [RSDevice] [realsense.py:324] [INFO] Setting Depth option "option.laser_power" to 360
[2023-10-20 17:46:12,932] [RSDevice] [realsense.py:411] [INFO] Running <RSDevice: hand_camera (Intel RealSense D435, S/N: 146322076186)> as a separate process
[2023-10-20 17:46:13,740] [RSDevice] [realsense.py:352] [INFO] Started device <RSDevice: hand_camera (Intel RealSense D435, S/N: 146322076186)> with 4 streams
[2

FEngine (64 bits) created at 0x55767f487c80 (threading is enabled)
FEngine resolved backend: OpenGL


[2023-10-20 17:46:17,217] [O3DGUIVisualizer] [o3d_gui_visualizer.py:1579] [INFO] Running <O3DGUIVisualizer: Point Clouds> as a separate process


<Camera: hand_camera (S/N: 146322076186) config={'Color': (848, 480, 60), 'Depth': (848, 480, 60), 'Infrared 1': (848, 480, 60), 'Infrared 2': (848, 480, 60)}>

In [2]:
image_obs = camera.get_images(True)
image_obs.keys()

dict_keys(['rgb', 'depth', 'ir_l', 'ir_r'])

In [3]:
tag = "60cm_max_power"

save_dir = Path("capture").resolve()
save_dir.mkdir(parents=True, exist_ok=True)

camera.take_picture()
np.savez(save_dir / f"{tag}_images.npz", **camera.get_images())
np.savez(save_dir / f"{tag}_params.npz", **camera.get_params())

In [4]:
visualizer.render()